In [1]:
########## import some useful package ##########

import time
t1 = time.time()

import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

import ripser
import persim

In [2]:
########## read the data ##########

processes = ["hhvv", "hhmumu", "jjBG_ptcut", "ttBG_ptcut", "bbBG_ptcut", "wwBG", "wwvvBG", "zzBG", "zzvvBG"]
event_type = [str(i)+"_500K" for i in processes]

Xsection = [4.0360050e-02, +3.3945676e-03, 1.345150e+01, 4.423600e+00, 2.325600e+00, 1.506969e+02, 1.224052e+01, 9.618000e+00, 6.473005e+00]     ### unit: fb
Luminosity = 1000   ### unit: fb^-1
simulation_num = 500000

features = ["VLCjetR10N2", "VLCjetR10N2.PT", "VLCjetR10N2.Eta", "VLCjetR10N2.Phi", "VLCjetR10N2.Mass", "MissingET.MET", "VLCjetR10N2.BTag", "Particle", "Particle.PID", "Particle.Status", "Particle.Eta", "Particle.Phi", "Particle.PT"]

##### set data path #####

event_path = []
for type in event_type:
    event_path.append("/data/mucollider/two_boosted/" + type + "/delphes_output.root")

##### get the data file #####

data_file =[]
for path in event_path:
    data_file.append(uproot.open(path))
    
##### read data with features #####
events = []     ### total events
for process in processes:
    tmp_events = []
    for feature in features:
        tmp_events.append(data_file[processes.index(process)]["Delphes;1"][feature].array())
    tmp_events = np.expand_dims(tmp_events, axis=-1)
    tmp_events = tmp_events.transpose((1,0,2))
    tmp_events = np.squeeze(tmp_events,axis=(2,))
    events.append(tmp_events)
    print("Time:{:^8.4f}(s)".format(time.time()-t1))
del tmp_events

Time:26.7532 (s)
Time:252.9913(s)
Time:487.6304(s)
Time:584.7951(s)
Time:648.1785(s)
Time:679.0967(s)
Time:708.6691(s)
Time:725.8660(s)
Time:753.0955(s)


In [3]:
########## define useful function ##########

##### calculate significance #####

def significance(s,b):
    return np.sqrt(2*((s+b)*np.log(1+s/b)-s))

##### count event number #####

def count(events):
    events_num = []
    for i, process in enumerate(processes):
        events_num.append(len(events[processes.index(process)]))
        print("There are", events_num[i], process, "events. Corresponding cross section:", Xsection[processes.index(process)]*events_num[i]/simulation_num, "(fb)")
        
##### select if Fat Jet >= 2 #####
        
def Fat_Jet_selection(events):
    where1 = np.where(events[:,features.index("VLCjetR10N2")]>=2)
    return events[where1]

##### select if M_jet_leading > XX GeV #####

def mass_leading_selection(events, low_mass_cut, high_mass_cut):
    where1 = []
    for i in range(len(events)):
        switch=1
        if events[i][features.index("VLCjetR10N2.Mass")][0]<low_mass_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.Mass")][0]>high_mass_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### select if M_jet_subleading > XX GeV #####

def mass_subleading_selection(events, low_mass_cut, high_mass_cut):
    where1 = []
    for i in range(len(events)):
        switch=1
        if events[i][features.index("VLCjetR10N2.Mass")][1]<low_mass_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.Mass")][1]>high_mass_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### select if pT_J_leading > XX GeV #####

def pT_leading_selection(events, low_pT_cut, high_pT_cut):
    where1 = []
    for i in range(len(events)):
        switch = 1
        if events[i][features.index("VLCjetR10N2.PT")][0]<low_pT_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.PT")][0]>high_pT_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### select if pT_J_subleading > XX GeV #####

def pT_subleading_selection(events, low_pT_cut, high_pT_cut):
    where1 = []
    for i in range(len(events)):
        switch = 1
        if events[i][features.index("VLCjetR10N2.PT")][1]<low_pT_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.PT")][1]>high_pT_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### define X_HH (defined in 2207.09602v2) #####

def calculate_X_HH(event):
    jet_mass1 = event[features.index("VLCjetR10N2.Mass")][0]
    jet_mass2 = event[features.index("VLCjetR10N2.Mass")][1]
    diff1 = np.abs(jet_mass1 - 124)
    diff2 = np.abs(jet_mass2 - 124)
    if diff1<diff2:
        m1 = jet_mass1
        m2 = jet_mass2
    else:
        m1 = jet_mass2
        m2 = jet_mass1
    return np.sqrt(((m1-124)/(0.1*m1+0.00001))**2 + ((m2-115)/(0.1*m2+0.00001))**2)

##### calculate M_JJ #####

def calculate_M_JJ(event):
    p = [0,0,0,0]    ### four momentum
    for i in range(2):    ### leading jet and sub-leading jet
        pt = event[features.index("VLCjetR10N2.PT")][i]
        eta = event[features.index("VLCjetR10N2.Eta")][i]
        phi = event[features.index("VLCjetR10N2.Phi")][i]
        mass = event[features.index("VLCjetR10N2.Mass")][i]
        p[1] = p[1] + pt*np.cos(phi)    ### px
        p[2] = p[2] + pt*np.sin(phi)    ### py
        p[3] = p[3] + pt*np.sinh(eta)    ### pz
        p[0] = p[0] + np.sqrt( mass**2 + (pt*np.cos(phi))**2 + (pt*np.sin(phi))**2 + (pt*np.sinh(eta))**2 )    ### energy
    M_JJ = np.sqrt(p[0]**2 - p[1]**2 - p[2]**2 - p[3]**2)
    return M_JJ

##### calculate pT_JJ #####

def calculate_pT_JJ(event):
    p = [0,0,0,0]    ### four momentum
    for i in range(2):    ### leading jet and sub-leading jet
        pt = event[features.index("VLCjetR10N2.PT")][i]
        eta = event[features.index("VLCjetR10N2.Eta")][i]
        phi = event[features.index("VLCjetR10N2.Phi")][i]
        mass = event[features.index("VLCjetR10N2.Mass")][i]
        p[1] = p[1] + pt*np.cos(phi)    ### px
        p[2] = p[2] + pt*np.sin(phi)    ### py
        p[3] = p[3] + pt*np.sinh(eta)    ### pz
        p[0] = p[0] + np.sqrt( mass**2 + (pt*np.cos(phi))**2 + (pt*np.sin(phi))**2 + (pt*np.sinh(eta))**2 )    ### energy
    pT_JJ = np.sqrt(p[1]**2 + p[2]**2)
    return pT_JJ

##### calculate \Delta_\Eta #####

def calculate_dEta_JJ(event):
    return np.abs(event[features.index("VLCjetR10N2.Eta")][0] - event[features.index("VLCjetR10N2.Eta")][1])

##### calculate TDA #####

def calculate_TDA(event):
    neutrino_ID = [12, 14, 16, -12, -14, -16]
    where_jet_particle = np.where(event[features.index("Particle.Status")]==1)[0]
    where_neutrino = np.where(np.isin(event[features.index("Particle.PID")], neutrino_ID))[0]
    where_jet_particle = np.delete(where_jet_particle, np.isin(where_jet_particle, where_neutrino))
    
    center_eta = (event[features.index("VLCjetR10N2.Eta")][0]+event[features.index("VLCjetR10N2.Eta")][1])/2    ### centerize the event
    center_phi = (event[features.index("VLCjetR10N2.Phi")][0]+event[features.index("VLCjetR10N2.Phi")][1])/2
    particle_eta = event[features.index("Particle.Eta")][where_jet_particle]-center_eta
    particle_phi = event[features.index("Particle.Phi")][where_jet_particle]-center_phi
    
    where_larger_pi = np.where(particle_phi>np.pi)
    particle_phi[where_larger_pi] = -(2*np.pi - particle_phi[where_larger_pi])    ### check if jet particle split
    where_smaller_mpi = np.where(particle_phi<-np.pi)
    particle_phi[where_smaller_mpi] = 2*np.pi + particle_phi[where_smaller_mpi]    ### check if jet particle split

    P = np.array([particle_eta, particle_phi]).T   ### calculate TDA
    diagrams = ripser.ripser(P)['dgms']
    
    return diagrams

##### calculate TDA sum of lifetime #####

def TDA_sum(diagrams):
    sum_l0 = np.sum(diagrams[0][:-1,1] - diagrams[0][:-1,0])
    sum_l1 = np.sum(diagrams[1][:,1] - diagrams[1][:,0])
    return sum_l0, sum_l1

##### calculate TDA mean of lifetime #####

def TDA_mean(diagrams):
    l0 = diagrams[0][:-1,1]-diagrams[0][:-1,0]
    l1 = diagrams[1][:,1]-diagrams[1][:,0]
    if np.sum(l1)==0:
        l1 = np.concatenate([l1,[0]])
    return np.mean(l0), np.mean(l1)

##### calculate TDA entropy #####

def TDA_entropy(diagrams):
    l0 = diagrams[0][:-1,1]-diagrams[0][:-1,0]
    L0 = np.sum(l0)
    S0 = (l0/L0)*np.log2((l0/L0))
    l1 = diagrams[1][:,1]-diagrams[1][:,0]
    L1 = np.sum(l1)
    S1 = (l1/L1)*np.log2((l1/L1))
    return -np.sum(S0), -np.sum(S1)

##### calculate TDA Lifetime*Birthtime #####

def TDA_LB(diagrams):
    return np.sum(diagrams[1][:,0]*(diagrams[1][:,1]-diagrams[1][:,0]))

In [4]:
########## preselection ##########

print("Before any selection:")
count(events)

##### 2 fat jet selection #####

for process in processes:
    events[processes.index(process)] = Fat_Jet_selection(events[processes.index(process)])
print("\nAfter 2 fat jet selection:")
count(events)

##### leading jet pT selection #####

leading_low_pT_cut = 200   ### GeV
leading_high_pT_cut = 800   ### GeV
for process in processes:
    events[processes.index(process)] = pT_leading_selection(events[processes.index(process)], leading_low_pT_cut, leading_high_pT_cut)
print("\nAfter leading jet pT selection:")
count(events)

##### subleading jet pT selection #####

subleading_low_pT_cut = 200   ### GeV
subleading_high_pT_cut = 600   ### GeV
for process in processes:
    events[processes.index(process)] = pT_subleading_selection(events[processes.index(process)], subleading_low_pT_cut, subleading_high_pT_cut)
print("\nAfter subleading jet pT selection:")
count(events)

##### leading jet mass selection #####

leading_low_mass_cut = 65
leading_high_mass_cut = 150
for process in processes:
    events[processes.index(process)] = mass_leading_selection(events[processes.index(process)], leading_low_mass_cut, leading_high_mass_cut)
print("\nAfter leading jet mass selection:")
count(events)

##### subleading jet mass selection #####

subleading_low_mass_cut = 65
subleading_high_mass_cut = 150
for process in processes:
    events[processes.index(process)] = mass_subleading_selection(events[processes.index(process)], subleading_low_mass_cut, subleading_high_mass_cut)
print("\nAfter subleading jet mass selection:")
count(events)

Before any selection:
There are 500000 hhvv events. Corresponding cross section: 0.04036005 (fb)
There are 500000 hhmumu events. Corresponding cross section: 0.0033945676 (fb)
There are 500000 jjBG_ptcut events. Corresponding cross section: 13.4515 (fb)
There are 500000 ttBG_ptcut events. Corresponding cross section: 4.4236 (fb)
There are 500000 bbBG_ptcut events. Corresponding cross section: 2.3256 (fb)
There are 500000 wwBG events. Corresponding cross section: 150.6969 (fb)
There are 500000 wwvvBG events. Corresponding cross section: 12.24052 (fb)
There are 500000 zzBG events. Corresponding cross section: 9.618 (fb)
There are 500000 zzvvBG events. Corresponding cross section: 6.473005 (fb)

After 2 fat jet selection:
There are 500000 hhvv events. Corresponding cross section: 0.04036005 (fb)
There are 500000 hhmumu events. Corresponding cross section: 0.0033945676 (fb)
There are 499999 jjBG_ptcut events. Corresponding cross section: 13.451473097 (fb)
There are 500000 ttBG_ptcut events

In [5]:
##### calculate the high level features #####

num_of_events = []

m_J_leading = []
m_J_subleading = []
pt_J_leading = []
pt_J_subleading = []
missET = []
X_HH = []
M_JJ = []
pT_JJ = []
dEta_JJ = []
BTag_leading = []
BTag_subleading = []

for process in processes:
    num_of_events.append(len(events[processes.index(process)]))
    for i in range(len(events[processes.index(process)])):
        m_J_leading.append(events[processes.index(process)][i][features.index("VLCjetR10N2.Mass")][0])
        m_J_subleading.append(events[processes.index(process)][i][features.index("VLCjetR10N2.Mass")][1])
        pt_J_leading.append(events[processes.index(process)][i][features.index("VLCjetR10N2.PT")][0])
        pt_J_subleading.append(events[processes.index(process)][i][features.index("VLCjetR10N2.PT")][1])
        missET.append(events[processes.index(process)][i][features.index("MissingET.MET")][0])
        X_HH.append(calculate_X_HH(events[processes.index(process)][i]))
        M_JJ.append(calculate_M_JJ(events[processes.index(process)][i]))
        pT_JJ.append(calculate_pT_JJ(events[processes.index(process)][i]))
        dEta_JJ.append(calculate_dEta_JJ(events[processes.index(process)][i]))
        BTag_leading.append(events[processes.index(process)][i][features.index("VLCjetR10N2.BTag")][0])
        BTag_subleading.append(events[processes.index(process)][i][features.index("VLCjetR10N2.BTag")][1])
    print("Time:{:^8.4f}(s)".format(time.time()-t1))

Time:843.2267(s)
Time:859.1850(s)
Time:863.2252(s)
Time:867.2677(s)
Time:871.7032(s)
Time:877.4444(s)
Time:888.7287(s)
Time:895.6385(s)
Time:909.2504(s)


In [6]:
##### calculate TDA high level features #####

H_0 = []    ### sum of 0'th order lifetime 
H_1 = []    ### sum of 1'th order lifetime 
S_0 = []    ### sum of 0'th order entropy
S_1 = []    ### sum of 1'th order entropy
LB_1 = []   ### sum of 1'th order Lifetime*Birthtime

for process in processes:
    for i in range(len(events[processes.index(process)])):
        if (i%20000)==19999:
            print("check point")
        TDA_tmp = calculate_TDA(events[processes.index(process)][i])
        H_0.append(TDA_sum(TDA_tmp)[0])        
        H_1.append(TDA_sum(TDA_tmp)[1])
        S_0.append(TDA_entropy(TDA_tmp)[0])   
        S_1.append(TDA_entropy(TDA_tmp)[1])
        LB_1.append(TDA_LB(TDA_tmp))
    print("Time:{:^8.4f}(s)".format(time.time()-t1))

check point
check point
check point
check point
check point
check point
check point
Time:1616.6655(s)
check point
check point
check point
check point
check point
check point
check point
Time:2488.7194(s)
check point
Time:2935.1650(s)
check point
Time:3562.7515(s)
check point
Time:4119.3166(s)
check point
check point
Time:4220.5603(s)
check point
check point
check point
check point
Time:4443.0378(s)
check point
check point
check point
Time:4591.9748(s)
check point
check point
check point
check point
check point
check point
Time:4906.9733(s)


In [8]:
##### make target #####

labels = np.arange(len(num_of_events))
target_everytype = np.repeat(labels, num_of_events)    ### this target 0~8 represent ["hhvv", "hhmumu", "jjBG_ptcut", "ttBG_ptcut", "bbBG_ptcut", "wwBG", "wwvvBG", "zzBG", "zzvvBG"]

target_sigbg = np.zeros(sum(num_of_events))
sig_num = num_of_events[0]+num_of_events[1]
target_sigbg[0:sig_num] = 1     ### this target 1 represent signal, 0 represent background

In [9]:
##### output test data #####

d = {'m_J_leading':m_J_leading, 'm_J_subleading':m_J_subleading, 'pt_J_leading':pt_J_leading, 'pt_J_subleading':pt_J_subleading, 'missET':missET, 'X_HH':X_HH, 'M_JJ':M_JJ, 'pT_JJ':pT_JJ, 'dEta_JJ':dEta_JJ, 'BTag_leading':BTag_leading, 'BTag_subleading':BTag_subleading, 'H_0':H_0, 'H_1':H_1, 'S_0':S_0, 'S_1':S_1, 'LB_1':LB_1, 'target_sigbg':target_sigbg, 'target_everytype':target_everytype}
df = pd.DataFrame(data=d)
df

,m_J_leading,m_J_subleading,pt_J_leading,pt_J_subleading,missET,X_HH,M_JJ,pT_JJ,dEta_JJ,BTag_leading,BTag_subleading,H_0,H_1,S_0,S_1,LB_1,target_sigbg,target_everytype
0,131.929962,106.218628,400.476990,237.498596,285.028748,1.022137,901.330146,296.698741,1.861570,7,7,11.616989,0.185851,5.478836,2.813458,0.107292,1.0,0
1,117.418617,104.973862,373.547333,336.402344,112.033386,1.107428,922.749668,112.863569,1.436828,6,6,13.186689,0.347521,5.659055,3.292051,0.105573,1.0,0
2,98.107506,106.901619,330.205902,325.322632,215.076736,2.350095,880.264486,212.935456,1.609447,5,7,1007.296177,0.098753,0.130734,3.095432,0.011124,1.0,0
3,125.354378,123.265610,335.786682,316.032196,108.490753,0.828154,823.209426,105.733273,1.289048,7,5,8.902533,0.133330,5.025661,2.081374,0.043093,1.0,0
4,128.480240,99.431831,291.614044,276.747559,85.948959,1.604073,612.402124,81.806900,0.269446,7,6,11.900936,0.273723,5.979420,2.286914,0.062588,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733548,68.598793,81.800018,371.462036,364.774902,212.044876,8.506933,861.356620,212.798192,1.192922,0,0,8.304975,0.188459,5.294803,2.617185,0.026724,0.0,8
733549,85.919716,89.774925,417.803955,315.517578,420.501343,5.097957,665.975073,421.582307,0.620782,0,0,6.711385,0.050603,4.827732,1.168970,0.019597,0.0,8
733550,75.380516,80.826782,423.029388,351.792816,101.512627,7.493714,833.692467,109.514224,0.726800,0,7,10.885185,0.202139,5.752752,3.325962,0.033762,0.0,8
733551,92.606163,89.427643,464.197845,331.032776,285.547577,4.435018,1259.716512,284.473227,2.127578,0,0,8.873080,0.280046,5.083157,3.011074,0.031015,0.0,8


In [10]:
h5f = h5py.File('/data/mucollider/two_boosted/data_HLfeature.h5', 'w')
for key, value in d.items():
    h5f.create_dataset(key, data=value)
    print(f"已寫入 Dataset: {key}")

已寫入 Dataset: m_J_leading
已寫入 Dataset: m_J_subleading
已寫入 Dataset: pt_J_leading
已寫入 Dataset: pt_J_subleading
已寫入 Dataset: missET
已寫入 Dataset: X_HH
已寫入 Dataset: M_JJ
已寫入 Dataset: pT_JJ
已寫入 Dataset: dEta_JJ
已寫入 Dataset: BTag_leading
已寫入 Dataset: BTag_subleading
已寫入 Dataset: H_0
已寫入 Dataset: H_1
已寫入 Dataset: S_0
已寫入 Dataset: S_1
已寫入 Dataset: LB_1
已寫入 Dataset: target_sigbg
已寫入 Dataset: target_everytype


In [11]:
print("Time:{:^8.4f}(s)".format(time.time()-t1))

Time:5061.3790(s)


In [12]:
h5f.close()